In [1]:
%%HTML
<style>
.container{width:70%;}
</style>

In [2]:
import time
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from IPython.core.display import clear_output
from shaolin.core.dashboard import Dashboard #pip install shaolin should make it work
from test_functions import TestFunctions
import copy

#from collections import deque


class TabuList(object):
    
    
    def __init__(self,pos=None,val=None,memory_size=100):
        """Class that manages the algorithm memory. The memory consists of the previous visited local minima.
            These local minima will act as a repeling force that forces the walkers to visit unexplored areas
        """
        
        self._Nd = len(pos)
        self.N = memory_size
        self.positions = np.tile(pos,(self.N,1))
        self.values = np.ones(self.N).reshape(self.N,1)*np.nan#nan for not used
        self.values[:2] = val
        self._pos_cand =[] if pos is None else [pos]
        self._val_cand = [] if val is None else [val]
        self.flows = np.zeros(self.N).reshape(self.N,1)
        
    
    def reset_cands(self):
        """This list acts like a buffer. When updating the content of the tabu list,
        the best point in the candidates list will be added to the tabu list"""
        self._pos_cand = [] 
        self._val_cand = []

    @property
    def alives(self):
        cond = np.logical_not(np.isnan(self.values))
        res = np.arange(len(cond))[cond.flatten()]
        alives = res.reshape(len(res),1)
        return alives.flatten()
    
    @property
    def max_cand(self):
        """Returns the position with the best associated value from the candidates list"""
        ix = self._val_cand.index(max(self._val_cand))
        return self._pos_cand[ix],self._val_cand[ix]
    
    def __len__(self):
        """Returns the length of the tabu list"""
        return len(self.values)
    
    def __repr__(self):
        """Prints the pairs (position,value) of the tabu list"""
        s = ""
        for i,pos in enumerate(self.positions):
            s += str(pos)+" : "+str(self.values[i])+"\n"
        return s
    
    def __getitem__(self, i):
        """Returns the position i in the tabu list"""
        return self.positions[i]
    
    def random_alive(self):
        """Returns the index that correspond to valid memories"""
        ix = np.random.choice(np.arange(len(self.alives)))
        return self.values[ix].flatten()
    
    def flow(self):
        """Flow metric analogous to the one used in the fractal taking into account the values of the tabu list 
            instead of the algorithm walkers. That means that the tabu list will act as some sort of "memory walkers"

        """
        min_pot = self.values[self.alives.flatten()].min()
        max_pot = self.values[self.alives.flatten()].max()
        #Calculate a normalized version of the potential of the positions contained in the tabu list
        unstarted_val = (np.ones(len(self.values[self.alives]))*-1).reshape(len(self.values[self.alives]),1)
        norm_val = ((self.values[self.alives]-min_pot)/(max_pot-min_pot))*0.9+0.1 
        
        self.scaled_pot = unstarted_val if max_pot==min_pot else norm_val
        self.scaled_pot = self.scaled_pot.reshape(len(self.alives),1)
        #Distance from a randomly chosen compaion. In this case the random companion is sampled as a permutation
        #of the original values, meaning that all the values in the list will be chosen for comparison
        companions = self.compas()
        res = self.positions-self.positions[companions]        
        self.distance = np.linalg.norm(res,axis=1).reshape(self.N,1)        
        self.flows[self.alives.flatten()] = self.distance[self.alives.flatten()]*(self.scaled_pot+1)
        
    
    
    def compas(self):
        """returns a random companion index for clone purposes"""
        return np.random.permutation(np.arange(self.N)).astype(int)
    
    def clone(self):
        """Decide if pairs of randomly chosen tabu list elements should be cloned"""
        #Random companions like in the flow function
        companions = self.compas()
        #probability of clone depends on the flow ratio between the two selected companions
        probs = np.minimum(1,np.maximum(0,(self.flows[companions]-self.flows)/np.maximum(1e-8,self.flows)))
        #will clone with probability p
        will_clone = np.random.random(size=len(self.values))<probs.flatten()
        #dont clone to unstarted memories
        for i,c in enumerate(companions):
            if np.isnan(self.values[c]):
                will_clone[i]=False
        #clone the selected values
        cloned = copy.copy(self.positions)
        values = copy.copy(self.values)
        for i,x in enumerate(will_clone.tolist()):
            if x:
                cloned[i,:] = self.positions[companions[i],:]
                values[i] = self.values[companions[i]]
        self.positions = cloned
        self.values = values
        
    
    def update_list(self):
        """Updates the tabu list elements with the best element from the candidates buffer.
            This update process will not only add one more element to the tabu list, but also will
            clone some values of the tabu list to a randomly chosen companion depending on its flow 
            ratio.
        """
        pos,val = self.max_cand
        self._append_tabu(pos,val)
        self.flow()
        self.clone()
        
        
        
    def append(self,pos,val):
        """Adds a new pair of (position,value) to the candidates list"""
        self._pos_cand.append(pos)
        self._val_cand.append(val)
       
    
    def _append_tabu(self,pos,val):
        """Updates the content of the tabu list with the selected (pos,val).
           This upodate will depend on the selected density value
        """
        if len(self.alives)==self.N:
            ix = np.random.randint(len(self.values))
        else:
            deads = np.arange(self.N)[np.isnan(self.values.flatten())]
            ix = np.random.choice(deads)
        
        self.positions[ix,:] = pos
        self.values[ix] = val

In [3]:
class Walkers(Dashboard):
    """Class in charge of iterating walkers"""
    
    def __init__(self,
                 pot_func,
                 alpha=1,
                 minjump=None,
                 max_temp=1,
                 mean_temp=-0.1,
                 min_temp=-1.,
                 tabu_len=10,
                 
                 num_walkers=150,
                 **kwargs
                ):
        def _end_callback():
            return False
        
        
        self.end_callback = _end_callback
        self.pot_func = pot_func
        
        
        dash = ["r$N=Fractal_opt",[["c$N=temp_col",["(-10.,10.,0.1,"+str(float(mean_temp))+")$d=Mean temp",
                                                    "(-10.,10.,0.1,"+str(float(min_temp))+")$d=min temp",
                                                    "(-10.,10.,0.1,"+str(float(max_temp))+")$d=Max temp"]
                                   ],["c$N=gaussian_col",["(1,10000,1,"+str(num_walkers)+")$d=Num walkers",
                                                          ["r$n=samp_opts",["True$d=Sample mean","False$d=Fractal LJ"]]
                                                         ]
                                   ],["c$N=generic_col",[
                                                      "(1,10000,1,"+str(tabu_len)+")$d=Tabu len",
                                                       ]
                                     ]
                                  ]
               ]
        Dashboard.__init__(self,dash,**kwargs) 
        self.reset()
   


    @property
    def best_so_far(self):
        return -self._best_so_far
    @best_so_far.setter
    def best_so_far(self,val):
        self._best_so_far = val
        
    @property
    def best_pos_so_far(self):
        return self.pot_func.unscale(self._best_pos_so_far)
    @best_pos_so_far.setter
    def best_pos_so_far(self,val):
        self._best_pos_so_far =val
    
   
    
    def reset(self):
        """Initializes the parameters of the algorithm"""
        #these are scaled
        positions = np.array([self.pot_func.to_scaled(self.pot_func.random_in_domain()) for _ in range(self.num_walkers.value)])
        self.positions = positions.reshape(self.num_walkers.value,len(self.pot_func.random_in_domain()))
        #init tracking of best solution
        self._best_so_far = -1e100
        self._best_pos_so_far = np.ones(len(self.positions[0]))*1000
        self.tabu_list = TabuList(self._best_pos_so_far,self._best_so_far)
        #init internal parameters
        self.jumps = np.ones(self.num_walkers.value)
        self.potentials = np.zeros(self.num_walkers.value)
        self.calculate_potential()
        if self.fractal_lj.value:
            self.best_pos = self.positions[0,:]
            self.max_pot = -1*self.pot_func.evaluate(self.pot_func.unscale(self.positions))[0]
        self._best_pos_so_far = self.best_pos
        self._best_so_far = self.max_pot
        #same jump scale forever
        minscale = self.mean_temp.value+self.min_temp.value
        maxscale = self.mean_temp.value+self.max_temp.value
        self.minjump = 10.0**minscale
        self.maxjump = 10.0**maxscale     
        #finish initialization
        self.calculate_jumps()
        self.tabu_list = TabuList(self._best_pos_so_far,self._best_so_far,memory_size=self.tabu_len.value)
        self.update_best()
        
       
       
    def _TTF(self,i):
        """Tabu list score for one walker"""
        a = self.positions[i,:]
        b = self.tabu_list.random_alive()
        c = np.linalg.norm(a-b)**2
        return 1+np.log(1+c)
    
    def tabu_tunneling(self):
        """Calculate the tabu list scores for all the walkers"""
        ttf = []
        for i in range(len(self.potentials)):
            ttf.append(self._TTF(i))
        return np.array(ttf).reshape(len(self.potentials),1)
        
        
    def calculate_potential(self):
        "Calculates function and tabu potentials for all the walkers"
        def fractal_lennard(U,N):
            U = U.reshape(N,3)
            npart = len(U)
            Epot = 0.0
            particles = np.random.choice(np.arange(npart),size=1,replace=False)
            for i in particles:
                samples = np.arange(npart)
                samples = samples[samples!=i]
                compas = np.random.choice(samples,size=2,replace=False)
                r1 = np.linalg.norm(U[compas[0],:]-U[i,:])**2
                p1_6 = r1*r1*r1
                p1_12 = p1_6*p1_6
                r2 = np.linalg.norm(U[compas[1],:]-U[i,:])**2
                p2_6 = r2*r2*r2
                Epot = p1_12-p2_6
            return Epot

        if self.fractal_lj.value:
            try:
                potential = -1*np.array([fractal_lennard(self.pot_func.unscale(self.positions)[i,:],self.pot_func.N) for i in range(self.positions.shape[0])])
                potential = potential.reshape(self.positions.shape[0],1)
            except:
                print("fallo random")#fails when using custom lennard jonnes
                raise
        else:
            #internally a maximization
            potential = -1*self.pot_func.evaluate(self.pot_func.unscale(self.positions))
        self.tabu_pot = self.tabu_tunneling()
        self.best_i = np.argmax(potential)
        min_pot = min(potential)
        max_pot = max(potential)
        #keep track of best
        if not self.fractal_lj.value:
            self.max_pot = max(self._best_so_far,max_pot)
            self.best_pos = self.positions[self.best_i,:] if self.max_pot != self._best_so_far else self._best_pos_so_far
        else:
            self.max_pot = max_pot
            self.best_pos = self.positions[self.best_i,:]
        #potentials scaled from 0.1 to 1
        new_pots = (((potential-min_pot)/(max_pot-min_pot))*0.9+0.1 if max_pot!=min_pot else np.ones(len(potential))*0.99).reshape(len(potential),1)
        self.potentials = new_pots
       

        
    def calculate_jumps(self):
        """Calculates the jump value for the random walk process"""
        self.jumps = self.minjump+(self.maxjump-self.minjump)*(1-self.potentials)
        self.jumps = self.jumps.reshape(len(self.jumps),1)
    
    
    def _local_search(self,pos):
        """Appends to the tabu list the result of a local search on pos and keeps track of it."""
        self.tabu_list.reset_cands()
        result = minimize(self.pot_func.evaluate,
                                  self.pot_func.unscale(pos),
                                  bounds=self.pot_func.domain,
                                  method="L-BFGS-B",
                                 )
        loc_pos = self.pot_func.to_scaled(result.x)
        loc_val = -result.fun
        self.tabu_list.append(loc_pos,loc_val)
        self.tabu_list.update_list()
        max_pos,max_val = self.tabu_list.max_cand  
        improves = max_val>self._best_so_far
        in_dom = self.pot_func.in_domain(self.pot_func.unscale(max_pos))
        if improves and in_dom:
            dif = np.linalg.norm(self._best_pos_so_far-max_pos)
            self._best_pos_so_far = max_pos
            self._best_so_far = max_val
        
    
    def update_best(self):
        """Ads to the tabu list a local optimization of the best walker on this epoch"""
        self._local_search(self.best_pos)
        if self.sample_mean.value:
            mean = (self.positions*self.potentials/self.potentials.sum()).sum(axis=0)
            self._local_search(mean)
  
       
        
    def gaussian_step(self):
        """Random perturbations of the positions of the walkers"""
        old_pos = self.positions.copy()
        pos_s = old_pos.shape
       
        dof = 2*np.random.random(size=pos_s)-1
        norm = dof/np.linalg.norm(dof)
        
        vel_mod = np.abs(np.array([np.random.normal(loc=0,scale=self.jumps[i]) for i in range(min(len(self.jumps),pos_s[0]))])).reshape(min(len(self.jumps),pos_s[0]),1)
        new_pos = old_pos+vel_mod*norm
        #If new position not in function domain, take a new step with half of the initial jump
        #until all the new position are in the function domain.
        for i in range(pos_s[0]):
            num_try=0
            while not self.pot_func.in_domain(self.pot_func.unscale(new_pos[i,:])) and num_try<100:
                dof = 2*np.random.random(size=pos_s[1])-1
                norm_i = dof/np.linalg.norm(dof)
                new_vel = np.abs(np.random.normal(loc=0,scale=self.jumps[i]/(2**num_try+1)))
                new_pos[i,:] = old_pos[i,:]+new_vel*norm_i
                vel_mod[i,:] = new_vel
                num_try +=1
                if num_try==100:
                    print(new_pos[i,:],new_vel)
                    raise
        self.vel  = vel_mod
        self.positions = new_pos
        self._old_pos = old_pos
        
        
    def step(self):
        "Perform one iteration of the algorithm"
        self.calculate_potential()
        self.calculate_jumps()
        self.update_best()
        self.gaussian_step()  

In [4]:
class FractalOptimizer(Dashboard):
    def __init__(self,
                 pot_func,
                 max_temp=1,
                 mean_temp=-0.2,
                 min_temp=-1.,
                 num_walkers=100,
                 max_epoch=100000,
                 tabu_len=100,
                 max_reads=50000,
                 **kwargs
                 ):
        walkers = Walkers(pot_func,
                          num_walkers=num_walkers,
                          name='walkers',
                          mean_temp=mean_temp,
                          min_temp=min_temp,
                          max_temp=max_temp,
                          tabu_len=tabu_len)
        dash = ["r$N=fractal_opt",[ "##Fractal$n=title",
                                   ["r$N=fractal_opt_controls",[
                                       ["c$N=fractal_col",["(0,1000000,1,"+str(max_epoch)+")$d=Max epoch",
                                                       "(0,10000000,1,"+str(max_reads)+")$d=Max reads",
                                                       "btn$d=Run&n=run_btn"
                                                        ]
                                       ],walkers]
                                  ]
               ]]
        Dashboard.__init__(self,dash,**kwargs)
        self.run_btn.observe(self.run)

    #Aliases and general properties of the fractal algorithm   
    @property
    def num_walkers(self):
        return self.walkers.num_walkers
    @property
    def pot_func(self):
        return self.walkers.pot_func
    @pot_func.setter
    def pot_func(self,val):
        self.walkers.pot_func = val
    
    @property
    def positions(self):
        return self.walkers.positions
    @positions.setter
    def positions(self,val):
        self.walkers.positions =val
    
    @property
    def potentials(self):
        return self.walkers.potentials
    @property
    def best_so_far(self):
        return self.walkers.best_so_far
    @property
    def best_pos_so_far(self):
        return self.walkers.best_pos_so_far
    @property
    def n_reads(self):
        return self.walkers.pot_func.n_reads
    
    @n_reads.setter
    def n_reads(self,val):
        self.walkers.pot_func.n_reads = val
        
    @property
    def step(self):
        return self.walkers.step
    
    @property
    def best(self):
        return (self.walkers.best_pos_so_far,self.walkers.best_so_far)
    
    def reset(self):
        """Resets the optimization process to its initial parameters"""
        self.epoch = 0
        self.n_reads = 0
        self.walkers.reset()
    
    def random_alives(self):
        """Return a list o length num_walkers containing the
        index of a diferent random walker in the function domain.
        """
        self.alive_ix = np.arange(self.walkers.num_walkers.value).tolist()
        #sampling with replacement of random alive companions
        alives = np.random.choice(self.alive_ix,size=(self.walkers.num_walkers.value))
        #ensure a walker doesnt compare to itself
        companions = [x+1 if alives[i]==i and i!=len(alives)-1 else x for i,x in enumerate(alives)]
        #if its comparing to itself pick the next one
        companions[-1] = companions[-1] if alives[-1]!=len(alives)-1 else companions[-2]
        return companions
        
        
    def flow(self):
        "calculate the flow metric of all the walkers"
        companions = self.random_alives()
        self.distance = np.linalg.norm(self.positions-self.positions[companions],axis=1).reshape(self.num_walkers.value,1)
        self.flows = self.distance*(self.potentials+1)*self.walkers.tabu_pot
        
    
    def clone(self):
        """Clone different walkers with a probability depending on the flow"""
        companions = self.random_alives()
        #comparing flows as (compa-walker)/walker bounded from 0 to 1
        probs = np.minimum(1,np.maximum(0,(self.flows[companions]-self.flows)/np.maximum(1e-8,self.flows)))
        #evaluate probabilities
        
        will_clone = np.random.random(size=self.num_walkers.value)<probs.flatten()
        #clone selected walkers
        cloned = copy.copy(self.positions)
        for i,x in enumerate(will_clone.tolist()):
            if x:
                cloned[i,:] = self.positions[companions[i],:]
        self.positions = cloned
        return cloned
    
    def run(self,_=None, end_callback=None):
        """Run the algorithm the selected number of iteration or until a custom callback condition is met"""
        #callback config
        def _end_callback():
            return False
        if end_callback is None:
            end_callback = _end_callback
        self.walkers.end_callback = end_callback
        #main loop
        self.reset()
        time_start = time.time()
        while not end_callback() and self.epoch<self.max_epoch.value and self.n_reads < self.max_reads.value:   
            self.step()
            self.flow()
            self.clone()
            self.epoch += 1


In [5]:
funcs = TestFunctions()

In [6]:
optimizer = FractalOptimizer(funcs.functions['eggholder'])

In [7]:
optimizer[0]#click run to perform optimization, ignore warnings

/home/kalidus/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:109: RuntimeWarning: invalid value encountered in less


In [8]:
optimizer.best# return X, val

(array([ 512.        ,  404.23180496]), -959.64066272085086)